In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from prophet import Prophet
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")

C:\Users\chris\anaconda4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\chris\anaconda4\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\chris\anaconda4\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Create a dataframe from the data
greece_inflation = pd.read_csv('Inflation and Parameters.csv')
greece_inflation.head()

,Date,Cpi,Food Price Index,Energy Price Index,Hicp,Unemployment Rate,Gdp,Rent Price,House Prices,Lci
0,2001-01-01,0.001806,0.016060,0.111474,0.003837,0.109312,0.000000,0.000000,0.315560,0.000000
1,2001-02-01,0.000000,0.011688,0.097947,0.000000,0.110324,0.005330,0.006099,0.328415,0.017084
2,2001-03-01,0.036626,0.008565,0.077760,0.042722,0.110324,0.010660,0.012198,0.341270,0.034167
3,2001-04-01,0.054424,0.003301,0.082858,0.063443,0.118421,0.015991,0.018297,0.354126,0.051251
4,2001-05-01,0.062677,0.009190,0.087208,0.072909,0.117409,0.029163,0.025907,0.352209,0.054301


In [3]:
# Rename the Date column to the appropriate name for the model implementation
greece_inflation.rename(columns = {greece_inflation.columns[0]:'ds'}, inplace = True)

In [4]:
greece_inflation.head()

,ds,Cpi,Food Price Index,Energy Price Index,Hicp,Unemployment Rate,Gdp,Rent Price,House Prices,Lci
0,2001-01-01,0.001806,0.016060,0.111474,0.003837,0.109312,0.000000,0.000000,0.315560,0.000000
1,2001-02-01,0.000000,0.011688,0.097947,0.000000,0.110324,0.005330,0.006099,0.328415,0.017084
2,2001-03-01,0.036626,0.008565,0.077760,0.042722,0.110324,0.010660,0.012198,0.341270,0.034167
3,2001-04-01,0.054424,0.003301,0.082858,0.063443,0.118421,0.015991,0.018297,0.354126,0.051251
4,2001-05-01,0.062677,0.009190,0.087208,0.072909,0.117409,0.029163,0.025907,0.352209,0.054301


In [5]:
# Select the appropriate columns
cols=['ds', 'Cpi', 'Hicp']
inflation_parameters = greece_inflation.loc[:, greece_inflation.columns.intersection(cols)]
inflation_parameters.head()

,ds,Cpi,Hicp
0,2001-01-01,0.001806,0.003837
1,2001-02-01,0.000000,0.000000
2,2001-03-01,0.036626,0.042722
3,2001-04-01,0.054424,0.063443
4,2001-05-01,0.062677,0.072909


In [6]:
inflation_parameters[['Cpi', 'Hicp']] = inflation_parameters[['Cpi', 'Hicp']].apply(pd.to_numeric, errors='coerce')
inflation_parameters.head()

,ds,Cpi,Hicp
0,2001-01-01,0.001806,0.003837
1,2001-02-01,0.000000,0.000000
2,2001-03-01,0.036626,0.042722
3,2001-04-01,0.054424,0.063443
4,2001-05-01,0.062677,0.072909


In [7]:
# Just in case there are nan values
inflation_parameters = inflation_parameters.fillna(method='ffill')
inflation_parameters = inflation_parameters.fillna(method='bfill')

In [8]:
# Just in case there are duplicate values
inflation_parameters = inflation_parameters[~inflation_parameters.index.duplicated(keep='first')]

In [9]:
# The Fb prophet model predicts univariate time series. It requires the Date index column to be named ds
# and the value column should be named y
inflation_parameters['ds'] = pd.DatetimeIndex(inflation_parameters['ds'])

In [10]:
inflation_parameters

,ds,Cpi,Hicp
0,2001-01-01,0.001806,0.003837
1,2001-02-01,0.000000,0.000000
2,2001-03-01,0.036626,0.042722
3,2001-04-01,0.054424,0.063443
4,2001-05-01,0.062677,0.072909
...,...,...,...
251,2021-12-01,0.841888,0.854694
252,2022-01-01,0.834924,0.849066
253,2022-02-01,0.864844,0.874137
254,2022-03-01,0.939902,0.945766


In [11]:
# Check for stationary
for col in ['Cpi', 'Hicp']:
    result = adfuller(inflation_parameters[col])
    print('p-value for ' + col + ' : %f' % result[1])

Check if the inflation parameters are stationary :
p-value for Cpi : 0.934648
p-value for Hicp : 0.874048


In [12]:
inflation_parameters['Cpi'] = inflation_parameters['Cpi'].diff()
inflation_parameters['Hicp'] = inflation_parameters['Hicp'].diff()
inflation_parameters.head()

,ds,Cpi,Hicp
0,2001-01-01,NaN,NaN
1,2001-02-01,-0.001806,-0.003837
2,2001-03-01,0.036626,0.042722
3,2001-04-01,0.017797,0.020721
4,2001-05-01,0.008254,0.009465


In [13]:
def fbprophet_predict(time_series, col):
    
    model = Prophet()
    
    # split data into train and test
    train_size = 0.8
    train = time_series[:int(train_size*(len(time_series)))]
    test = time_series[int(train_size*(len(time_series))):]
    
    # model fit to the train data
    model.fit(train)
    
    # value prediction for each index of the test dataset
    forecast = model.predict(test[['ds']])
    
    # ds is the daily index, yhat is the forecast, yhat_lower and yhat_upper are
     # the lower and upper bound of the predicted value. 
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    # calculate metrics to evaluate the prediction
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for parameter[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for parameter[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for parameter[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [14]:
# rename columns to appropriate names
cpi_parameter = inflation_parameters[['ds', 'Cpi']]
cpi_parameter.rename(columns = {'Cpi':'y'}, inplace = True)
cpi_parameter = cpi_parameter.iloc[1: , :]
cpi_parameter.head()

,ds,y
1,2001-02-01,-0.001806
2,2001-03-01,0.036626
3,2001-04-01,0.017797
4,2001-05-01,0.008254
5,2001-06-01,-0.002063


In [15]:
cpi_parameter['y'].isnull().values.any()

False

In [16]:
hicp_parameter = inflation_parameters[['ds', 'Hicp']]
hicp_parameter.rename(columns = {'Hicp':'y'}, inplace = True)
hicp_parameter = hicp_parameter.iloc[1: , :]
hicp_parameter.head()

,ds,y
1,2001-02-01,-0.003837
2,2001-03-01,0.042722
3,2001-04-01,0.020721
4,2001-05-01,0.009465
5,2001-06-01,-0.001791


In [17]:
hicp_parameter['y'].isnull().values.any()

False

In [18]:
datasets = [cpi_parameter, hicp_parameter]

In [19]:
inflation_indexes = ['Cpi', 'Hicp']

i=0

# apply the model for each parameter
for data in datasets:
    test, forecast, error = fbprophet_predict(data, inflation_indexes[i])
    
    month_test = pd.Series(test['y'])
    month_test.index = test['ds']
    month_test.index = pd.to_datetime(month_test.index)
    
    month_forecast = pd.Series(forecast['yhat'])
    month_forecast.index = forecast['ds']
    month_forecast.index = pd.to_datetime(month_forecast.index)
    
    print("Monthly prediction error for " + str(inflation_indexes[i]) + ":\n")
    
    print("MAE: " + str(mean_absolute_error(month_test.resample("1m").mean().bfill(), month_forecast.resample("1m").mean().bfill())))
    print("MSE: " + str(mean_squared_error(month_test.resample("1m").mean().bfill(), month_forecast.resample("1m").mean().bfill())))
    print("RMSE: " + str(math.sqrt(mean_squared_error(month_test.resample("1m").mean().bfill(), month_forecast.resample("1m").mean().bfill()))))
    
    
    i = i+1

10:32:10 - cmdstanpy - INFO - Chain [1] start processing
10:32:10 - cmdstanpy - INFO - Chain [1] done processing
10:32:10 - cmdstanpy - INFO - Chain [1] start processing
10:32:10 - cmdstanpy - INFO - Chain [1] done processing


          ds      yhat  yhat_lower  yhat_upper
0 2018-02-01 -0.025650   -0.040579   -0.011072
1 2018-03-01  0.050628    0.035103    0.066311
2 2018-04-01  0.009841   -0.004990    0.024190
3 2018-05-01 -0.005166   -0.020667    0.009403
4 2018-06-01 -0.004209   -0.018735    0.009794
          ds         y
0 2018-02-01 -0.001290
1 2018-03-01  0.038174
2 2018-04-01  0.023730
3 2018-05-01 -0.008512
4 2018-06-01  0.025793
FB Prophet RMSE for parameter[Cpi]: 0.020277215970058337

FB Prophet MSE for parameter[Cpi]: 0.0004111654874963888

FB Prophet MAE for parameter[Cpi]: 0.016311225523318592

Monthly prediction error for Cpi:

MAE: 0.016311225523318592
MSE: 0.0004111654874963888
RMSE: 0.020277215970058337
          ds      yhat  yhat_lower  yhat_upper
0 2018-02-01 -0.027193   -0.042032   -0.012155
1 2018-03-01  0.049875    0.035409    0.064710
2 2018-04-01  0.010881   -0.004973    0.025725
3 2018-05-01 -0.001808   -0.016680    0.012019
4 2018-06-01 -0.002569   -0.018159    0.012726
          

## Split based on Season

Each index has a seasonal cycle of one year or 365 days.

In [20]:
cpi_parameter

,ds,y
1,2001-02-01,-0.001806
2,2001-03-01,0.036626
3,2001-04-01,0.017797
4,2001-05-01,0.008254
5,2001-06-01,-0.002063
...,...,...
251,2021-12-01,0.018313
252,2022-01-01,-0.006964
253,2022-02-01,0.029920
254,2022-03-01,0.075058


In [21]:
def fbprophet_predict_seasonal(time_series, col):
    
    model = Prophet()
    time_series.index = time_series['ds']
    
    # split by year
    train, test = time_series['2001-02-01' : '2019-12-31'], time_series['2020-01-01' : '2022-04-01']
    
    # fit the model to the train data
    model.fit(train)
    
    # value prediction for each index of the test dataset
    forecast = model.predict(test[['ds']])
    
    # ds is the daily index, yhat is the forecast, yhat_lower and yhat_upper are
     # the lower and upper bound of the predicted value.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    # calculate metrics to evaluate the prediction
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Athens[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Athens[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Athens[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [22]:
inflation_indexes = ['Cpi', 'Hicp']

i=0
# apply the model for each parameter
for data in datasets:
    test, forecast, error = fbprophet_predict_seasonal(data, inflation_indexes[i])
    i = i+1

10:32:11 - cmdstanpy - INFO - Chain [1] start processing
10:32:11 - cmdstanpy - INFO - Chain [1] done processing
10:32:11 - cmdstanpy - INFO - Chain [1] start processing
10:32:11 - cmdstanpy - INFO - Chain [1] done processing


          ds      yhat  yhat_lower  yhat_upper
0 2020-01-01 -0.025534   -0.042004   -0.009212
1 2020-02-01 -0.021955   -0.037665   -0.006597
2 2020-03-01  0.048796    0.031899    0.064717
3 2020-04-01  0.012212   -0.003882    0.027501
4 2020-05-01 -0.005467   -0.021152    0.011485
          ds         y
0 2020-01-01 -0.044622
1 2020-02-01 -0.012897
2 2020-03-01  0.040753
3 2020-04-01 -0.011865
4 2020-05-01 -0.021150
FB Prophet RMSE for Athens[Cpi]: 0.021447470956817673

FB Prophet MSE for Athens[Cpi]: 0.0004599940104435376

FB Prophet MAE for Athens[Cpi]: 0.016982767032375227

          ds      yhat  yhat_lower  yhat_upper
0 2020-01-01 -0.024624   -0.039970   -0.009370
1 2020-02-01 -0.022359   -0.037949   -0.007145
2 2020-03-01  0.047848    0.031544    0.063022
3 2020-04-01  0.013490   -0.002024    0.029266
4 2020-05-01 -0.000810   -0.015821    0.013426
          ds         y
0 2020-01-01 -0.033768
1 2020-02-01 -0.009210
2 2020-03-01  0.031722
3 2020-04-01 -0.003070
4 2020-05-01 -0.004

## Yearly predictions

In [23]:
cols=['ds', 'Cpi', 'Hicp']

inflation_parameters = greece_inflation.loc[:, greece_inflation.columns.intersection(cols)]
inflation_parameters.head()

,ds,Cpi,Hicp
0,2001-01-01,0.001806,0.003837
1,2001-02-01,0.000000,0.000000
2,2001-03-01,0.036626,0.042722
3,2001-04-01,0.054424,0.063443
4,2001-05-01,0.062677,0.072909


In [24]:
inflation_parameters.index = inflation_parameters['ds']

In [25]:
inflation_parameters = inflation_parameters.drop(['ds'],axis=1)

In [26]:
inflation_parameters.index = pd.to_datetime(inflation_parameters.index)

In [27]:
# convert data to annual period
parameters_yearly = inflation_parameters.resample("Y").mean().bfill()

In [28]:
parameters_yearly['ds'] = parameters_yearly.index

In [29]:
# annual data concerning inflation indexes
parameters_yearly

,Cpi,Hicp,ds
ds,,,
2001-12-31,0.047094,0.054596,2001-12-31
2002-12-31,0.116478,0.128422,2002-12-31
2003-12-31,0.186441,0.195915,2003-12-31
2004-12-31,0.245916,0.257035,2004-12-31
2005-12-31,0.320759,0.329731,2005-12-31
2006-12-31,0.390616,0.401232,2006-12-31
2007-12-31,0.455915,0.467873,2007-12-31
2008-12-31,0.552274,0.565000,2008-12-31
2009-12-31,0.581571,0.597233,2009-12-31


In [30]:
# rename columns to appropriate names
cpi_parameter = parameters_yearly[['ds', 'Cpi']]
cpi_parameter.rename(columns = {'Cpi':'y'}, inplace = True)
cpi_parameter.head()

,ds,y
ds,,
2001-12-31,2001-12-31,0.047094
2002-12-31,2002-12-31,0.116478
2003-12-31,2003-12-31,0.186441
2004-12-31,2004-12-31,0.245916
2005-12-31,2005-12-31,0.320759


In [31]:
cpi_parameter['y'].isnull().values.any()

False

In [32]:
# rename columns to appropriate names
hicp_parameter = parameters_yearly[['ds', 'Hicp']]
hicp_parameter.rename(columns = {'Hicp':'y'}, inplace = True)
hicp_parameter.head()

,ds,y
ds,,
2001-12-31,2001-12-31,0.054596
2002-12-31,2002-12-31,0.128422
2003-12-31,2003-12-31,0.195915
2004-12-31,2004-12-31,0.257035
2005-12-31,2005-12-31,0.329731


In [33]:
datasets = [cpi_parameter, hicp_parameter]

In [34]:
inflation_indexes = ['Cpi', 'Hicp']

i=0
# apply the model for each parameter
for data in datasets:
    test, forecast, error = fbprophet_predict(data, inflation_indexes[i])
    
    i = i+1

10:32:12 - cmdstanpy - INFO - Chain [1] start processing
10:32:12 - cmdstanpy - INFO - Chain [1] done processing
10:32:12 - cmdstanpy - INFO - Chain [1] start processing


          ds      yhat  yhat_lower  yhat_upper
0 2018-12-31  0.950464    0.831364    1.082347
1 2019-12-31  1.005624    0.892360    1.145513
2 2020-12-31  1.034289    0.905591    1.157302
3 2021-12-31  1.085907    0.956441    1.221836
4 2022-12-31  1.139300    1.015189    1.271433
          ds         y
0 2018-12-31  0.740242
1 2019-12-31  0.746797
2 2020-12-31  0.714212
3 2021-12-31  0.745766
4 2022-12-31  0.909917
FB Prophet RMSE for parameter[Cpi]: 0.2763861162090175

FB Prophet MSE for parameter[Cpi]: 0.07638928523310452

FB Prophet MAE for parameter[Cpi]: 0.271729964859993



10:32:13 - cmdstanpy - INFO - Chain [1] done processing


          ds      yhat  yhat_lower  yhat_upper
0 2018-12-31  0.966552    0.850753    1.091347
1 2019-12-31  1.021470    0.905306    1.133513
2 2020-12-31  1.054092    0.935876    1.172671
3 2021-12-31  1.104881    0.981048    1.227286
4 2022-12-31  1.157737    1.038666    1.279275
          ds         y
0 2018-12-31  0.779654
1 2019-12-31  0.793148
2 2020-12-31  0.760062
3 2021-12-31  0.774921
4 2022-12-31  0.917242
FB Prophet RMSE for parameter[Hicp]: 0.260855021031388

FB Prophet MSE for parameter[Hicp]: 0.0680453419972859

FB Prophet MAE for parameter[Hicp]: 0.25594097440131847

